In [ ]:
import os
import pickle

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from experiments.logisticRegression.utils import multilogistic_get_tgt_log_density as get_tgt_log_density
from experiments.logisticRegression.mnist.load_mnist import mnist_dataset_full
from variational.exponential_family import GenericMeanFieldNormalDistribution, MeanFieldNormalDistribution

predictors, labels = mnist_dataset_full(return_test=False, path_prefix="..")
K = labels.shape[1]
N, dim = predictors.shape
latent_dim = dim * K
# Gaussian Prior
my_prior_covariance = 25 * jnp.identity(latent_dim)
# my_prior_covariance = my_prior_covariance.at[0, 0].set(400)
my_prior_covariance = jnp.diag(my_prior_covariance)
my_prior_log_density = MeanFieldNormalDistribution(jnp.zeros(latent_dim), my_prior_covariance).log_density
tgt_log_density = get_tgt_log_density(predictors, labels, my_prior_log_density)

# Mean Field Gaussian Variational Family
my_variational_family = GenericMeanFieldNormalDistribution(dimension=latent_dim)

PKLs = []
PKL_titles = []
for file in os.listdir("./"):
    if file.endswith(".pkl"):
        PKLs.append(pickle.load(open(file, "rb")))
        PKL_titles.append(str(file))

full_gaussian = GenericMeanFieldNormalDistribution(dimension=latent_dim)

lossesPKLs = []
lossesPKL_titles = []
for file in os.listdir("./losses/"):
    if file.endswith(".pkl"):
        lossesPKLs.append(pickle.load(open("./losses/" + file, "rb")))
        lossesPKL_titles.append(str(file))